Verinin anlatılması, açıklanması.

In [ ]:
#EDA - Keşifsel Veri Analizi

Veriden elde edilen sonuçların eklenmesi, değerlendirilmesi.

In [ ]:
#Data Preprocessing- Veri Ön İşleme

Data üzerinde neler yapıldı, neden yapıldı açıklanması.

In [ ]:
#Algoritma seçimi ve Hiperparametre Optimizasyonu

Hangi algoritma neden seçildi, seçilen 2 algoritmadan hangisi nasıl sonuç çıkardı neden iyi/kötü sonuç çıkardı.

In [ ]:
#Model Evaluation

Model üzerinde değerlendirme algoritmaları kullanarak sonuçların karşılaştırılması.